# Launch a new instance of ASReview

Make sure to add your own authenication token from [ngrok.com](https://dashboard.ngrok.com/auth) to the cell below.

This notebook can be found at
[JTeijema/asreview-google-colab](https://github.com/JTeijema/asreview-google-colab).
This page has instructions on how to use this notebook.

The link to the launched instance can be found at [Start ASReview](#start).

In [ ]:
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "" #@param {type:"string"}
set_password_access = False #@param {type:"boolean"}
username = "asreview" #@param {type:"string"}
password = "colab" #@param {type:"string"}

## Install ASReview and dependencies

In [ ]:
# installing packages... Colab has a lot of packages pre-installed and might
# throw an exception if you try to install new packages. This can be ignored.

%pip install pyngrok --quiet
%pip install --pre --upgrade asreview --quiet

# remove sample folder
!rm -r sample_data

In [ ]:
# imports
from pyngrok import ngrok
import os
import asreview

## Mount a google drive
To store the data and results of ASReview on your google drive, you will need to
mount your drive. The cells below mount your drive to the folder
`/content/drive/MyDrive/ASReview/`.

In [ ]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive/')
! mkdir /content/drive/MyDrive/ASReview-colab

In [ ]:
# change ASReview project path to user files
os.environ['ASREVIEW_PATH'] = "/content/drive/MyDrive/ASReview-colab"

# if you do not want to use the google drive, you can use the following line instead
# os.environ['ASREVIEW_PATH'] = "/content/asreview"

## Set up ASReview
The cell below will set up ngrok and activate a tunnel for ASReview. 

In [ ]:
# set ngrok authtoken
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
# Terminate open tunnels if exist
ngrok.kill()

In [ ]:
# Connect to the tunnel
if set_password_access:
    session = ngrok.connect(port="80", auth=username+":"+password)
else:
    session = ngrok.connect(port="80")

<a name="start"></a>
## Start ASReview
The last cell will continue to run for as long as ASReview is active.

The ASReview session can be opened by clicking on the link in cell below. The
output of this cell will look like this: `Public URL:
http://xx-xx-xx-xx.ngrok.io/`.

If prompted, click `visit site`.

In [ ]:
# Get the public URL
print("Public URL:", session.public_url)

In [ ]:
!asreview lab --port 80

## Import ASReview files
Google blocks directly injecting files through the upload. This means that you
cannot upload `.asreview` files to ASReview. If you want to upload a project, we
will need to extract the file to the `ASReview-colab` folder. To use the cell
below, remove the `%%script echo... line and run the cell`. This will
start a file upload dialog. Select the `.asreview` file you want to upload and 
click `Open`. The file will be extracted to the `ASReview-colab` folder. You can
then open the project in ASReview.

>Note: the .asreview file should have the same name as the project itself, or ASReview will not be able to access the project.

Remember, to run any cell, you will first need to close the cell running ASReview.

In [ ]:
# extra imports
from google.colab import files
import zipfile
from pathlib import Path

In [ ]:
%%script echo To use this cell, remove this line

# Upload your file to the runtime
last_uploaded = list(files.upload().keys())[-1]
print("Uploaded file:", last_uploaded)

extract_path = "/content/drive/MyDrive/ASReview-colab/"+ Path(last_uploaded).stem

with zipfile.ZipFile(last_uploaded, 'r') as zip_ref:
    print("Extracting file...")
    zip_ref.extractall(extract_path)

print("Your project has been extracted to the project folder and should be available in ASReview.")
print("Removing left over file...")
! rm $last_uploaded